<a href="https://colab.research.google.com/github/OsamaHaikal/Cross-deep-model-/blob/master/deepleaeningcompetition_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost
from lightgbm import LGBMClassifier
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, InputLayer
from tensorflow.keras.layers import StringLookup, IntegerLookup
from keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from tensorflow.keras.callbacks import EarlyStopping
import string
from tensorflow.estimator import DNNClassifier

In [ ]:
dtypes = {
    'id' : 'float32',
    'bin_0' :'float32',    
    'bin_1' :'float32',    
    'bin_2': 'float32', 
    'ord_0': 'float32',
    'day': 'float32',
    'month': 'float32',
    'target': 'float32',
}

In [ ]:
num_cols = ['id',
'bin_0',
'bin_1',
'bin_2',
'day',
'ord_0',
'month',
'target',
]


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/cat-in-the-dat/train.csv", dtype=dtypes)
test_df = pd.read_csv("/content/drive/MyDrive/cat-in-the-dat/test.csv", dtype=dtypes)

In [ ]:
train_df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0.0,0.0,0.0,0.0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2.0,Grandmaster,Cold,h,D,kr,2.0,2.0,0.0
1,1.0,0.0,1.0,0.0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1.0,Grandmaster,Hot,a,A,bF,7.0,8.0,0.0
2,2.0,0.0,0.0,0.0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1.0,Expert,Lava Hot,h,R,Jc,7.0,2.0,0.0
3,3.0,0.0,1.0,0.0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1.0,Grandmaster,Boiling Hot,i,D,kW,2.0,1.0,1.0
4,4.0,0.0,0.0,0.0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1.0,Grandmaster,Freezing,a,R,qP,7.0,8.0,0.0


In [ ]:
train_df = train_df.drop(['id', 'ord_0', "nom_8", "nom_9"], axis=1)
test_df = test_df.drop(['id', 'ord_0', "nom_8", "nom_9"], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_df, train_df["target"], test_size=0.25, random_state=42)

In [ ]:
def df_to_dataset(dataframe=x_train, shuffle=True, batch_size=256, test=False):
    df = dataframe.copy()
    
    if not test:
        labels = df.pop('target')
    
    df = {
        key: value[:,tf.newaxis] 
        for key, value in dataframe.items()
    }
    
    if not test:
        ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    else:
        ds = tf.data.Dataset.from_tensor_slices((dict(df)))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    
    return ds

In [ ]:
train_dataset = df_to_dataset(x_train)
val_dataset = df_to_dataset(x_val, shuffle=False)
final_test_ds = df_to_dataset(test_df, shuffle=False ,test=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  if __name__ == '__main__':


# Deep Learning


In [ ]:
class FeatureEngineering:

	def __init__(
		  self, 
		  dataframe, 
		  target=None
		):
		self.dataframe = dataframe

	def select_columns(self):

		self.num_features = self.dataframe.select_dtypes(include=['float32']).columns
		self.cat_features = self.dataframe.select_dtypes(include=['object']).columns
		self.all_features = self.num_features.to_list() + self.cat_features.to_list()

		self.bin_cols = self.cat_features[self.cat_features.str.contains("bin_3|bin_4")].to_list()
		self.ord_cols = self.cat_features[self.cat_features.str.contains("ord")].to_list()
		self.nom_cols = self.cat_features[self.cat_features.str.contains("nom")].to_list()
  
		return self.all_features

	def create_input_layer(self):

		self.inputs = {
			feature: keras.layers.Input(
				  name=feature,
				  shape=(), 
				  dtype=tf.float32 if feature in self.num_features else tf.string
			)
			for feature in self.all_features
		}

		return self.inputs

	def encode_features(self):
		
		self.encoded_features = []

		for feature in self.cat_features:

			if feature in self.ord_cols:
				vocabulary = list(string.ascii_lowercase) + list(string.ascii_uppercase)
				lookup = StringLookup(
					vocabulary=vocabulary,
					output_mode="one_hot",
				)
			else:
				vocabulary  = list(self.dataframe[feature].unique())
				lookup = StringLookup(
					vocabulary=vocabulary,
					output_mode="count",
				)

			encoded_feature = lookup(tf.expand_dims(self.inputs[feature], -1))
			self.encoded_features.append(encoded_feature)

		for feature in self.num_features:

			encoded_feature = tf.expand_dims(self.inputs[feature], -1)
			self.encoded_features.append(encoded_feature)

		self.all_features_encoded = keras.layers.concatenate(self.encoded_features)
		return self.all_features_encoded

In [ ]:
train_df = train_df.drop(["target"], axis=1)

In [ ]:
fe = FeatureEngineering(train_df, target="target")
features_columns = fe.select_columns()
input_layer = fe.create_input_layer()
all_features_encoded = fe.encode_features()

In [ ]:
class DeepCrossModel:

	def __init__(
		self,
		inputs,
		all_features_encoded, 
		hidden_units,
		dropout_rate,
		learning_rate,
		optimizer,
		metrics,
		loss,
		**fit_params: dict,
	):
		self.inputs = inputs
		self.all_features_encoded = all_features_encoded
		self.hidden_units = hidden_units
		self.inputs = inputs
		self.dropout_rate = dropout_rate
		self.learning_rate = learning_rate
		self.optimizer = optimizer
		self.metrics = metrics
		self.loss = loss
		self.fit_params = fit_params

	def create_wide_network(self):
		self.wide = self.all_features_encoded
	
	def create_cross_network(self):

		self.cross = self.all_features_encoded
		for _ in self.hidden_units:
			self.neurons = self.cross.shape[-1]
			x = keras.layers.Dense(self.neurons, activation="relu", kernel_initializer='uniform')(self.cross)
			self.cross = keras.layers.Dropout(self.dropout_rate)(self.cross)
			self.cross = self.all_features_encoded * x + self.cross
   
	def create_deep_network(self):
		
		 self.deep = self.wide
		 for units in self.hidden_units:
				self.deep = keras.layers.Dense(
						units, 
						activation="relu",
						kernel_initializer='uniform', 
						)(self.deep)
				self.deep = keras.layers.Dense(
						units,
					  	activation="relu",
						kernel_initializer='uniform', 
				)(self.deep)
				self.deep = keras.layers.Dropout(self.dropout_rate)(self.deep)
				self.deep = keras.layers.BatchNormalization()(self.deep)

	def merge_networks(self):

		self.merged = keras.layers.concatenate([self.wide, self.deep], name="merged_layer")

	def create_output_layer(self):
		self.bn = keras.layers.BatchNormalization()(self.merged)
		self.outputs = keras.layers.Dense(1, activation="sigmoid")(self.bn)

	def create_model(self):
		
		self.model = keras.Model(inputs=self.inputs, outputs=self.outputs)

	def compile_model(self):

		self.model.compile(
			optimizer=self.optimizer,
			loss=self.loss,
			metrics=self.metrics,
		)

	def plot_model(self):
		keras.utils.plot_model(self.model, show_shapes=True, rankdir="LR")
	
	def run(self, *args, **kwargs):
		self.create_wide_network()
		#self.create_cross_network()
		self.create_deep_network()
		self.merge_networks()
		self.create_output_layer()
		self.create_model()
		self.compile_model()
		self.plot_model()

	def fit(self):
		
		self.history = self.model.fit(
			**self.fit_params,
		)
  
		return self.history
	
	def predict(self, x_test):

		self.predictions = self.model.predict(x_test)
  
		return self.predictions


	def fit_predict(self, x_test):
		
		self.fit()
		self.predict(x_test)

		return self.history, self.predictions

In [ ]:
learning_rate = 1e-4
decay_rate = learning_rate / 180
momentum = 0.9
sgd = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
adam = keras.optimizers.Adam(learning_rate=1e-4)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
tf.keras.backend.clear_session()

fit_params = dict(
    x=train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=tf.keras.callbacks.EarlyStopping(patience=8, monitor="val_auc"),
)

dcm = DeepCrossModel(
    inputs=input_layer,
    all_features_encoded=all_features_encoded,
    hidden_units=[32,16,8],
    dropout_rate=0.15,
    learning_rate=1e-5,
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC()],
    **fit_params,
)

In [ ]:
dcm.run()

In [ ]:
history = dcm.fit()

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['target'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


879/879 [==============================] - 31s 30ms/step - loss: 0.6090 - auc: 0.6159 - val_loss: 0.5838 - val_auc: 0.6687
Epoch 2/50
879/879 [==============================] - 25s 26ms/step - loss: 0.5572 - auc: 0.7077 - val_loss: 0.5650 - val_auc: 0.7012
Epoch 3/50
879/879 [==============================] - 28s 29ms/step - loss: 0.5484 - auc: 0.7219 - val_loss: 0.5577 - val_auc: 0.7098
Epoch 4/50
879/879 [==============================] - 25s 27ms/step - loss: 0.5447 - auc: 0.7272 - val_loss: 0.5556 - val_auc: 0.7153
Epoch 5/50
879/879 [==============================] - 25s 27ms/step - loss: 0.5421 - auc: 0.7313 - val_loss: 0.5555 - val_auc: 0.7123
Epoch 6/50
879/879 [==============================] - 26s 27ms/step - loss: 0.5397 - auc: 0.7349 - val_loss: 0.5543 - val_auc: 0.7165
Epoch 7/50
879/879 [==============================] - 26s 27ms/step - loss: 0.5370 - auc: 0.7384 - val_loss: 0.5560 - val_auc: 0.7137
Epoch 8/50
879/879 [==============================] - 28s 29ms/step - los

In [ ]:
preds = dcm.predict(final_test_ds)